### csv 파일 출처
* [나이스 교육정보 개방 포털](https://open.neis.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=OPEN13920190311110530306647&infSeq=1#none)

In [5]:
import requests
import json

# 위도 경도 구하기 ---- kako api
def get_coordinate(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'cba6eb29be7c48e97d9a5cd246429cdd'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

### 행정구역_지도
![행정구역_지도](../img/%ED%96%89%EC%A0%95%EA%B5%AC%EC%97%AD_%EC%A7%80%EB%8F%84.png)
* [출처 | 위키백과](https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%ED%96%89%EC%A0%95_%EA%B5%AC%EC%97%AD)

In [6]:
# 광역시, 특별시, 특별자치시, 특별자치도 - 다른 행정구역에 소속되지 않음 ( 광역시와 행정구역을 분리하여 생각 )

# 1특별시, 6광역시, 1특별자치시, 8도, 1특별자치도
# => 17개의 데이터 파일 생성 

# html 이름은 csv 파일의 '교육청'빼고 설정

In [7]:
# 학교 정보 크롤링(네이버)
import requests
from bs4 import BeautifulSoup

error = []

def crawling_high_school_data(high_school_name) -> list:

    url = f'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query={high_school_name}'
 
    response = requests.get(url)

    html = response.text

    soup = BeautifulSoup( html, 'html.parser' )

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    a_tag = soup.select('.home') # 홈페이지 주소
    data  = soup.select('dd') # 주소, 전화번호, 개교 연도, 학생수 등

    # 리스트에 담음
    list_data = []
    for i in data[:4]:
        list_data.append(i.text)

    # 홈페이지 주소 추가
    link = []
    for i in a_tag:
        link.append(i.attrs['href'])
        
    list_data.append(link[0])

    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    return list_data
    

In [8]:
crawling_high_school_data('대정고등학교')

['제주특별자치도 서귀포시 대정읍 일주서로2498번길 21',
 ' 064-730-0100 ',
 '1952년 06월 02일',
 '293명 남 293명, 여 0명 ',
 'http://daejeong.jje.hs.kr']

In [9]:
import pandas as pd

def put_high_school_data(high_school_name, affiliations) -> dict:
    high_school_data = crawling_high_school_data(high_school_name)
    
    # index
    # 0 : 주소
    # 1 : 전화번호
    # 2 : 개교 연도
    # 3 : 학생수
    # 4 : 홈페이지
    address = high_school_data[0]
    phone = high_school_data[1].strip(' ')
    birth = high_school_data[2]

    # index
    # 0 : 총 학생수
    # 2 : 남 학생수
    # 4 : 여 학생수
    students_data = high_school_data[3].split(' ')

    # 0 : 전체 학생수 / 2 : 남학생 수 / 4 : 여학생 수
    if len(students_data) <= 2: # 하지만 split한 list의 길이가 2 이하이면 총 학생수만
        total_students = students_data[0]
        boys            = '0명'
        girls           = '0명'
    else:
        total_students = students_data[0]
        boys            = students_data[2].strip(',')
        girls           = students_data[4]

    # 경도, 위도
    longitude, latitude = get_coordinate(high_school_data[0])
    
    link = high_school_data[4]

    result = {
        '주소' : address,
        '위도' : latitude,
        '경도' : longitude,
        '전화번호' : phone,
        '재학생 수' : total_students,
        '남학생 수' : boys,
        '여학생 수' : girls,
        '홈페이지' : link,
        '계열' : affiliations
    }

    return result

    

In [10]:
put_high_school_data('대정고등학교', '일반계')

{'주소': '제주특별자치도 서귀포시 대정읍 일주서로2498번길 21',
 '위도': 33.2319799989913,
 '경도': 126.259500374588,
 '전화번호': '064-730-0100',
 '재학생 수': '293명',
 '남학생 수': '293명',
 '여학생 수': '0명',
 '홈페이지': 'http://daejeong.jje.hs.kr',
 '계열': '일반계'}

In [11]:

# 딕셔너리에 집어넣고 csv 파일 만듦

import os
import pandas as pd

path = '../code/교육청_csv/'

file_list = os.listdir(path) # 폴더 안 파일 목록 가져옴

offices_of_education_csv = file_list

for i in offices_of_education_csv:
    df = pd.read_csv(path + i)

    high_school_names = list(set(df['학교명']))

    high_schools_data = {}

    for j in high_school_names:
        # 학교 계열
        _list = list(df[df['학교명'] == j]['계열명']) # 특정 학교 계열명 추출
        affiliations = ''
        try:
            for k in _list:
                affiliations += k + ' '
                if _list[-1] != k:
                    affiliations += ' | '
                    
            high_schools_data[j] = put_high_school_data(j, affiliations) 
        except:
            error.append(j)
            continue
            
    high_school_dataframe = pd.DataFrame(high_schools_data).T
    csv_name = i.strip('교육청.csv') + '고등학교데이터.csv'
    csv_path = '../data_csv/' + csv_name
    high_school_dataframe.to_csv(csv_path)


In [12]:
len(error)

470

In [13]:
# csv 파일 재설정 
# - unnamed: 0열 지우기
# - 다른 지역들에 같은 고등학교 때문에 고등학교 이름 앞에 행정구역의 이름을 붙여준다.
#    ->  그다음 다시 데이터를 집어넣어 값을 수정

data_csv_path = '../data_csv/'

file_list = os.listdir(data_csv_path) # 폴더 안 파일 목록 가져옴

data_csv_list = file_list

ero = []

for i in data_csv_list:
    df = pd.read_csv(data_csv_path + i)
    
    # Unnamed: 0열을 지우고 재설정
    save_index = list(df['Unnamed: 0'])
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df.index = save_index

    # 이름 앞에 행정구역의 이름 붙여줌
    head_word = i.strip('고등학교데이터.csv')
    
    high_school_addresses = list(df['주소'])

    high_school_names = df.index
    
    for j in range(len(high_school_addresses)):
        if head_word not in high_school_addresses[j]:
            df.rename(index={high_school_names[j] : head_word + ' ' + high_school_names[j]}, inplace=True)

            print(head_word + ' ' + high_school_names[j])

    df.to_csv(data_csv_path + i) # 저장

강원도 석정여자고등학교
강원도 인제고등학교
강원도 설악고등학교
강원도 성수고등학교
강원도 상동고등학교
강원도 대성고등학교
강원도 대진고등학교
강원도 대화고등학교
경기도 중산고등학교
경기도 광남고등학교
경기도 청담고등학교
경기도 홍익디자인고등학교
경기도 양진중학교
경기도 인창고등학교
경기도 대신고등학교
경기도 광문고등학교
경기도 오산고등학교
경기도 용문고등학교
경기도 진영고등학교
경기도 영신여자고등학교
경기도 한광고등학교
경기도 성남고등학교
경기도 백암고등학교
경기도 기흥고등학교
경상남도 한일여자고등학교
경상남도 영산고등학교
경상남도 덕산고등학교
경상남도 세종고등학교
경상남도 웅천고등학교
경상남도 동원고등학교
경상남도 용남고등학교
경상남도 영진고등학교
경상남도 경상고등학교
경상남도 대곡고등학교
경상남도 고성고등학교
경상남도 명덕고등학교
경상북도 청암고등학교
경상북도 유성여자고등학교
경상북도 세화고등학교
경상북도 경안고등학교
경상북도 장산초등학교병설유치원
경상북도 영양여자고등학교
경상북도 율곡고등학교
경상북도 금천고등학교
경상북도 영동고등학교
경상북도 풍산고등학교
경상북도 영덕고등학교
경상북도 영일고등학교
경상북도 경일고등학교
경상북도 세명고등학교
경상북도 대영고등학교
광주광역시 성덕중학교
광주광역시 광덕고등학교
광주광역시 동명고등학교
광주광역시 보문고등학교
광주광역시 상일여자고등학교
광주광역시 성덕고등학교
광주광역시 광주고등학교
광주광역시 금호고등학교
대구광역시 운암고등학교
대구광역시 수성고등학교
대구광역시 구암고등학교
대구광역시 경화여자고등학교
대구광역시 경신고등학교
대구광역시 강동고등학교
대구광역시 오성고등학교
대구광역시 대진고등학교
대구광역시 영신고등학교
대구광역시 대원고등학교
대구광역시 계성고등학교
대전광역시 계룡중학교
대전광역시 중일고등학교
대전광역시 서일고등학교
대전광역시 혜천여자고등학교
대전광역시 대덕여자고등학교
대전광역시 대성여자상업고등학교
부산광역시 영산고등학교
부산광역시 금성고등학교
부산광역시 경원고등학교
부산광역시 신도고등학교
부산광역시 광명

In [14]:
f = pd.read_csv('../data_csv/강원도고등학교데이터.csv')

list(f['주소'])

['강원도 원주시 치악로 1384',
 '강원도 횡성군 둔내면 둔내로 22',
 '강원도 홍천군 내면 구룡령로 5174',
 '인천광역시 남동구 방축로469번길 27',
 '강원도 춘천시 신북읍 신샘밭로 363-12',
 '강원도 동해시 청운로 112',
 '강원도 홍천군 서석면 구룡령로 2570',
 '강원도 평창군 봉평면 기풍4길 27-6',
 '강원도 춘천시 남면 여우내길 20',
 '강원도 춘천시 동면 장학길 24',
 '강원도 원주시 개운4길 25',
 '강원도 강릉시 용지로 107',
 '강원도 삼척시 근덕면 삼척로 3700',
 '강원도 속초시 배움터길 16',
 '강원도 횡성군 갑천면 갑천로 17',
 '강원도 태백시 상철암길 10-32',
 '강원도 춘천시 동면 만천양지길 95',
 '강원도 정선군 여량면 여량7길 42',
 '강원도 횡성군 횡성읍 교항로 33',
 '강원도 강릉시 강릉대로587번길 68',
 '강원도 원주시 원문로137번길 34',
 '강원도 강릉시 화부산로8번길 37',
 '강원도 강릉시 주문진읍 학교길 26',
 '강원도 철원군 동송읍 이평로123번길 40',
 '강원도 춘천시 학당길 28',
 '강원도 원주시 배울로 227',
 '강원도 인제군 남면 신남로 47-35',
 '강원도 강릉시 구정면 칠봉로 670-34',
 '강원도 인제군 기린면 창암길 20',
 '강원도 춘천시 후만로24번길 45',
 '강원도 횡성군 횡성읍 횡성로 252',
 '강원도 영월군 북면 문학로길 25-8',
 '강원도 삼척시 원덕읍 삼척로 480',
 '인천광역시 남동구 석산로197번길 19',
 '경기도 가평군 설악면 한양부로 21-1',
 '강원도 강릉시 강릉대로547번길 14',
 '강원도 춘천시 자라우새길 126',
 '강원도 삼척시 사직로 22',
 '강원도 동해시 효자남길 51',
 '강원도 영월군 주천면 주천로89번길 41',
 '강원도 춘천시 동산면 새술막길 630',
 '강원도 화천군 화천읍 상승로 69',
 '강원도 정선군 임

In [15]:
# 로고 주소 공식
# 홈페이지 주소 + / + img src
